# Алгоритм (кратко, одной ячейкой)

**Цель:** восстановить позиции пропусков (пробелов) в слепленной строке.

**Идея:** динамическое программирование по униграммной языковой модели.
1. Загружаем частоты слов RU/EN. `logP(w) = log(freq/total)`.
2. Эвристики: спец-термины (бренды/модели), числа, пунктуация, дефисы/тире, версии `x.y`. Язык подстроки — по алфавиту.
3. DP: для каждой позиции `i` ищем лучший разрез `j<i` (не дальше `max_word_len`), максимизируя сумму лог-вероятностей.
4. Из `prev` восстанавливаем границы и возвращаем **индексы** пробелов.

In [1]:
import math
import requests
import os
import re
from collections import defaultdict
import string

In [2]:
# Символы пунктуации
punct = string.punctuation + '-…«»'
attaching_punct = set(punct) - {'-'}

# Список "особых" терминов
special_terms = [
    'USSR', 'ЕГЭ', 'iPhone', 'HUAWEI', 'Huawei', 'Playstation', 'PLAYSTATION', 'Philips', 'HP', 'Samsung', 
    'Fender', 'Xbox One', 'Xiaomi', 'Indesit', 'Merida', 'LG', 'Yamaha', 'Ikea', 
    'Asus', 'Dyson', 'Stels', 'Oppo', 'Honor', 'Bosch', 'Adidas', 'TP-Link', 'JBL', 
    'Levi’s', 'Canon', 'Pixel', 'MSI', 'Gibson', 'Atlant', 'TCL', 'Lenovo', 'Sharp', 
    'JVC', 'Acer', 'Dell', 'ROG', 'Toshiba', 'Mac', 'OLED', 'LG OLED', 'A54', 'XR', 'RTX', 
    'Avito', 'Яндекс Еда', 'Сбербанк', 'шкаф-купе', 'стиралок', 'вайфай', '5.1',
    'Samsung A54', 'вайфай 6'
]

# Создание mapping для специальных терминов (concatenated версии)
special_mapping = {}
for t in special_terms:
    low_t = t.lower()
    concat = re.sub(r'[\s-]', '', low_t)
    special_mapping[concat] = t


## Загрузка словаря частот

In [3]:
def get_word_freq(lang='ru'):
    if lang == 'ru':
        url = "https://raw.githubusercontent.com/hermitdave/FrequencyWords/master/content/2018/ru/ru_full.txt"
    elif lang == 'en':
        url = "https://raw.githubusercontent.com/hermitdave/FrequencyWords/master/content/2018/en/en_full.txt"
    else:
        raise ValueError("Unsupported language")
    
    response = requests.get(url)
    if response.status_code == 200:
        lines = response.text.splitlines()
        word_freq = {}
        for line in lines:
            parts = line.split(' ')
            if len(parts) == 2:
                word, freq = parts
                word_freq[word.lower()] = int(freq)
        return word_freq
    else:
        raise Exception(f"Failed to download {lang} frequency list")

print("Загружаю словари...")
word_freq_ru = get_word_freq('ru')
total_freq_ru = sum(word_freq_ru.values())
max_freq_ru = max(word_freq_ru.values()) if word_freq_ru else 1

word_freq_en = get_word_freq('en')
total_freq_en = sum(word_freq_en.values())
max_freq_en = max(word_freq_en.values()) if word_freq_en else 1

Загружаю словари...


## Вероятность отдельного слова

In [4]:
def get_unigram_logp(word, word_freq, total_freq, max_freq):
    alpha = 0.05
    epsilon = 1e-9
    if word in special_mapping:
        return math.log(max_freq * 1e10 / total_freq)
    if word in word_freq:
        return math.log(word_freq[word] / total_freq)
    elif word.isdigit():
        return math.log(max_freq * 10 / total_freq)
    elif len(word) > 0 and all(c in punct for c in word):
        return math.log(max_freq * 50 / total_freq)
    
    # Обработка составных слов с дефисами или тире
    for sep in ['-']:
        if sep in word:
            parts = word.split(sep)
            parts = [p for p in parts if p]
            if len(parts) > 1 and all(p and (p in word_freq or len(p) > 1 or p.isdigit()) for p in parts):
                sub_logps = [get_unigram_logp(p, word_freq, total_freq, max_freq) for p in parts]
                sep_logp = math.log(max_freq * 50 / total_freq)  # как для пунктуации
                logp = sum(sub_logps) + (len(parts) - 1) * sep_logp + math.log(1.01)  # небольшой бонус для объединения
                return logp
    
    # Обработка чисел с точкой (версии, типа 5.1)
    if '.' in word:
        parts = word.split('.')
        parts = [p for p in parts if p]
        if len(parts) > 1 and all(p.isdigit() for p in parts):
            sub_logps = [get_unigram_logp(p, word_freq, total_freq, max_freq) for p in parts]
            sep_logp = math.log(max_freq * 50 / total_freq)
            logp = sum(sub_logps) + (len(parts) - 1) * sep_logp + math.log(1.01)
            return logp
    
    return math.log(epsilon) + (len(word) - 1) * math.log(alpha)

## Восстановление пробелов (DP с униграммами) и возврат индексов

In [5]:
def restore_spaces_indices(text: str) -> list:
    orig_text = text
    text = text.lower()
    n = len(text)
    max_word_len = 25
    
    dp = [float("-inf")] * (n+1)
    prev = [-1] * (n+1)
    prev_word = [""] * (n+1)
    dp[0] = 0.0
    
    for i in range(1, n+1):
        for j in range(max(0, i-max_word_len), i):
            substr = text[j:i]
            if any(1040 <= ord(c) <= 1103 for c in substr):
                wf, tf, mf = word_freq_ru, total_freq_ru, max_freq_ru
            else:
                wf, tf, mf = word_freq_en, total_freq_en, max_freq_en
            
            word = substr
            logp = get_unigram_logp(word, wf, tf, mf)
            
            score = dp[j] + logp
            if score > dp[i]:
                dp[i] = score
                prev[i] = j
                prev_word[i] = word
    

    indices = []
    i = n
    while i > 0:
        j = prev[i]
        if j > 0:  # Добавляем индекс только если это не начало строки
            indices.append(j)
        i = j
    indices.reverse() 
    return indices

In [6]:
# Обработка файла
def process_file(input_file: str, output_file: str):
    with open(input_file, 'r', encoding='utf-8') as infile:
        lines = infile.readlines()
        output_data = []
        for line in lines[1:]:
            line = line.strip()
            if not line:
                continue
            parts = line.split(',', 1)
            if len(parts) != 2:
                continue
            id_, text_no_spaces = parts
            indices = restore_spaces_indices(text_no_spaces)
            output_data.append(f'{id_},"{str(indices)}"')
    
    with open(output_file, 'w', encoding='utf-8') as outfile:
        outfile.write("id,predicted_positions\n")
        for text in output_data:
            outfile.write(text + '\n')

In [7]:
input_file = "dataset_1937770_3.txt"
output_file = "output.csv"

if not os.path.exists(input_file):
    raise FileNotFoundError(f"Файл {input_file} не найден")

process_file(input_file, output_file)
print(f"Файл сохранён как {output_file}")

Файл сохранён как output.csv
